In [ ]:
# ! pip install reportlab pdfplumber
# PDFの場合は左下が原点(0,0)
# 参考：https://livelifeulove.hatenablog.com/entry/2016/11/04/python%E3%81%A7ReportLab%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9FPDF%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E4%BD%9C%E3%82%8A%E6%96%B9%EF%BC%88%E3%83%81%E3%83%BC%E3%83%88%E7%9A%84%E3%81%AA%EF%BC%89
# 参考：

In [17]:
# mainの処理
# TODO:日本語の改行を適切に行う、qrコードのgenerative AIで、ロゴを追加
import qrcode
import random, json

from reportlab.lib.pagesizes import B5
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.enums import TA_CENTER
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import reportlab.lib, reportlab.platypus

from PIL import Image as PILImage

global hit_count

with open("./hit_count.json", "r") as f:
    hit_count = json.load(f)["count"]

probability = 1
max_hit = 100


text_ex = "斎宮歴史博物館	三重県	特別展「海の祈り－海浜の神社と伊勢神宮－」	2023/10/7	2023/11/26	https://www.bunka.pref.mie.lg.jp/saiku/50304036130.htm	島国である我が国において、人やモノ、文化などの交流、移動は、古くから海路を介することで行われていました。海路による交流や移動は、私たちの想像を超える困難を伴うものであったことは、さまざまな歴史資料などからも想像するに難くありません。世界遺産「神宿る島」宗像（むなかた）・沖ノ島と関連遺産群や八代（やつしろ）神社（三重県鳥羽市）「伊勢神島祭祀遺物」などに見られるように、人々が航海の無事や安全などを神々に祈願していたことは自然な流れでしょう。　この特別展では、これらの遺産や遺物をはじめ、明治時代初めまで行われていた伊勢神宮の贄海神事（にえうみしんじ）に関する資料などから人知の及ばぬ存在に対する畏怖や地域の歴史、神々への日々の安寧を願う想い、感謝の気持ちを表す祭祀や風習などを紹介します。	"

text_ex = text_ex.split("	")

museum_name = text_ex[0]
exhibition_name = text_ex[2]

exhibition_description = text_ex[6]
url = text_ex[5]

url_path = "./image/qr.png"
backgpund_path = "./image/material_201006_01_white.png"
backgpund_rote_path = "./image/material_201006_01_white_rotate.png"
logo_path = "./image/logo.png"
sealing_path = "./image/sealing.png"

font_name = "genshingothic"
font_path = "./font/genshingothic-20150607/GenShinGothic-Normal.ttf"


# フォントの登録
pdfmetrics.registerFont(TTFont(font_name, font_path))

# スタイルの取得
styles = getSampleStyleSheet()

def get_style(font_name, font_size, leading):
    return ParagraphStyle(
        "CenteredStyle",
        fontName=font_name,
        fontSize=font_size,
        leading=leading,
        alignment=TA_CENTER
    )

def background_image(canvas, doc):
    img_width, img_height = 100, 100
    # translateとrotateの関係で、左上、左下、右上、右下の順に画像を配置する

    # 左上
    canvas.saveState()
    canvas.translate(0, B5[1] - img_height)
    canvas.rotate(0)
    canvas.drawImage(backgpund_path, 0, 0, width=img_width, height=img_height)
    canvas.restoreState()

    # 左下
    canvas.saveState()
    canvas.translate(img_width, 0)
    canvas.rotate(90)
    canvas.drawImage(backgpund_path, 0, 0, width=img_width, height=img_height)
    canvas.restoreState()

    # 右上
    canvas.saveState()
    canvas.translate(B5[0] - img_width, B5[1])
    canvas.rotate(-90)
    canvas.drawImage(backgpund_path, 0, 0, width=img_width, height=img_height)
    canvas.restoreState()

    # 右下
    canvas.saveState()
    canvas.translate(B5[0], img_width)
    canvas.rotate(180)
    canvas.drawImage(backgpund_path, 0, 0, width=img_width, height=img_height)
    canvas.restoreState()


    # logoの配置
    # どこに配置するかは考える必要がある
    canvas.saveState()
    canvas.translate(0, 0)
    canvas.drawImage(logo_path, (B5[0]-58.26*3) // 2, 10, width=58.26*3, height=28.74*3)
    canvas.restoreState()

    # 当選機能の追加 当選の場合はシーリングスタンプを押す
    if random.random() < probability and hit_count < max_hit:
        canvas.saveState()
        canvas.translate(60, B5[1] - 180)
        canvas.rotate(30)
        canvas.drawImage(sealing_path, 0, 0, width=100, height=100)
        canvas.restoreState()
        # hit_count += 1
        # with open("./hit_count.json", "w") as f:
        #     json.dump({"count": hit_count}, f)


story = []
doc = SimpleDocTemplate("sample.pdf", pagesize=B5)

story.append(Paragraph("あなたへのおすすめは...", get_style(font_name, 20, 20)))

story.append(Spacer(1, 20))

story.append(Paragraph(museum_name, get_style(font_name, 30, 30)))

story.append(Spacer(1, 40))

story.append(Paragraph(exhibition_name, get_style(font_name, 30, 30)))

story.append(Spacer(1, 30))

story.append(Paragraph(exhibition_description, get_style(font_name, 10, 15)))

story.append(Spacer(1, 20))

#QRコードの設定
qr = qrcode.QRCode(
    version=2, #QRコードのバージョン(1~40)
    error_correction=qrcode.constants.ERROR_CORRECT_H #誤り訂正レベル(L：約7%,M：約15%,Q:約25%,H:約30%)
)
qr.add_data(url)
qr.make()
img = qr.make_image()
img.save(url_path)

story.append(Image(url_path, width=100, height=100))

story.append(Spacer(1, 40))


centered_style = get_style(font_name, 20, 20)


# ドキュメントに追加
doc.build(story, onFirstPage=background_image, onLaterPages=background_image)


In [9]:
# 参考：https://zenn.dev/micchi/articles/e0a584deeced0a
# ! pip install budoux
import budoux

parser = budoux.load_default_japanese_parser()

text = parser.parse(exhibition_description)

# max_length以下の文字数で分割
def combine_elements_until_max_length(data, max_length):
    if not data:
        return []

    combined_list = []
    current_string = ""

    for item in data:
        if len(current_string + item) <= max_length:
            current_string += item
        else:
            combined_list.append(current_string)
            current_string = item

    # 最後のcurrent_stringをリストに追加
    if current_string:
        combined_list.append(current_string)

    return combined_list


# 使用例
result = combine_elements_until_max_length(text, 43)
print("\n".join(result))

島国である我が国において、人やモノ、文化などの交流、移動は、古くから海路を介することで
行われていました。海路による交流や移動は、私たちの想像を超える困難を伴うものであった
ことは、さまざまな歴史資料などからも想像するに難くありません。世界遺産「神宿る島」宗像
（むなかた）・沖ノ島と関連遺産群や八代（やつしろ）神社（三重県鳥羽市）
「伊勢神島祭祀遺物」などに見られるように、人々が航海の無事や安全などを神々に
祈願していたことは自然な流れでしょう。　この特別展では、これらの遺産や遺物をはじめ、
明治時代初めまで行われていた伊勢神宮の贄海神事（にえうみしんじ）に関する資料などから
人知の及ばぬ存在に対する畏怖や地域の歴史、神々への日々の安寧を願う想い、感謝の気持ちを
表す祭祀や風習などを紹介します。


In [141]:
import qrcode

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, portrait
from reportlab.lib.units import cm

text_ex = "斎宮歴史博物館	三重県	特別展「海の祈り－海浜の神社と伊勢神宮－」	2023/10/7	2023/11/26	https://www.bunka.pref.mie.lg.jp/saiku/50304036130.htm	島国である我が国において、人やモノ、文化などの交流、移動は、古くから海路を介することで行われていました。海路による交流や移動は、私たちの想像を超える困難を伴うものであったことは、さまざまな歴史資料などからも想像するに難くありません。世界遺産「神宿る島」宗像（むなかた）・沖ノ島と関連遺産群や八代（やつしろ）神社（三重県鳥羽市）「伊勢神島祭祀遺物」などに見られるように、人々が航海の無事や安全などを神々に祈願していたことは自然な流れでしょう。　この特別展では、これらの遺産や遺物をはじめ、明治時代初めまで行われていた伊勢神宮の贄海神事（にえうみしんじ）に関する資料などから人知の及ばぬ存在に対する畏怖や地域の歴史、神々への日々の安寧を願う想い、感謝の気持ちを表す祭祀や風習などを紹介します。	"

text_ex = text_ex.split("	")


pdfmetrics.registerFont(TTFont(font_name, "C:/Windows/Fonts/HGRGE.TTC"))
pdfmetrics.registerFont(TTFont("HGRME", "C:/Windows/Fonts/HGRME.TTC"))

# A4の新規PDFファイルを作成
page = canvas.Canvas("sample.pdf", pagesize=portrait(A4))

page_center = A4[0]//2
page.setFont(font_name, 20)

page.drawCentredString(page_center, 720, "あなたへのおすすめは ...")

page.setFont(font_name, 20)
page.drawCentredString(page_center, 600, text_ex[0])
page.drawCentredString(page_center, 500, text_ex[2])



# 改行で表示する
# テキストオブジェクトを作成
text_object = page.beginText()
# テキストの開始位置を設定
text_object.setTextOrigin(100, 400)
# フォントとフォントサイズを設定
text_object.setFont(font_name, 10)
# テキストを追加
text = text_ex[-1]
length = 40
text_object.textLines("\n".join([text[i:i+length] for i in range(0, len(text), length)]))
# テキストオブジェクトを描画
page.drawText(text_object)


#QRコードの設定
qr = qrcode.QRCode(
    version=2, #QRコードのバージョン(1~40)
    error_correction=qrcode.constants.ERROR_CORRECT_H #誤り訂正レベル(L：約7%,M：約15%,Q:約25%,H:約30%)
)
qr.add_data(text_ex[-2])
qr.make()
img = qr.make_image()
img.save("qr.png")
img_width, img_height = img.size

img_width = img_width / 3
img_height = img_height / 3

# 画像を中心に配置するための座標を計算
x = (A4[0] - img_width) / 2

# 画像を中心に描画
page.drawImage("./qr.png", x, 100, img_width, img_height)

# PDFファイルとして保存
page.save()

img = qrcode.QRCode(
    version=2, #QRコードのバージョン(1~40)
    error_correction=qrcode.constants.ERROR_CORRECT_H #誤り訂正レベル(L：約7%,M：約15%,Q:約25%,H:約30%)
)
img.add_data(text_ex[-2])
img.make()
img.make_image()


In [121]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4

def draw_centred_wrapped_text(c, text, x, y, width, font, size):
    lines = []
    words = text.split()
    while words:
        line = ''
        while words and (c.stringWidth(line + words[0], font, size) <= width):
            line += (words.pop(0) + ' ')
        lines.append(line)
    
    # 中央揃えでテキストを描画
    total_height = (len(lines) - 1) * size
    y_position = y + (total_height / 2)
    for line in lines:
        c.drawCentredString(x, y_position, line.strip())
        y_position -= size

# 例
c = Canvas("example.pdf", pagesize=A4)
c.setFont("Helvetica", 12)
text = "これはサンプルの長いテキストです。自動的に改行しながら中央揃えで描画されるべきです。"
draw_centred_wrapped_text(c, text, A4[0]/2, A4[1]/2, 300, "Helvetica", 12)
c.save()

KeyboardInterrupt: 

In [90]:
qr = qrcode.QRCode(
    version=2, #QRコードのバージョン(1~40)
    error_correction=qrcode.constants.ERROR_CORRECT_H #誤り訂正レベル(L：約7%,M：約15%,Q:約25%,H:約30%)
)
qr.add_data(text_ex[-2])
qr.make()
img = qr.make_image()
type(img)


qrcode.image.pil.PilImage

In [42]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas

pdfmetrics.registerFont(TTFont(font_name, "C:/Windows/Fonts/HGRGE.TTC"))
pdfmetrics.registerFont(TTFont("HGRME", "C:/Windows/Fonts/HGRME.TTC"))

def create_pdf(output_path):
    # Canvasオブジェクトを作成
    c = canvas.Canvas(output_path, pagesize=A4)

    # テキストの追加
    c.setFont(font_name, 16)
    c.drawString(3*cm, A4[1] - 3*cm, "あなたへのクイズスタートは...")

    # 円形の図形を追加
    for i in range(3):
        c.circle(5*cm, A4[1] - (5 + i*2)*cm, 0.5*cm)

    # 三角形の図形を追加
    # for i in range(3):
    #     triangle = [(6*cm, A4[1] - (5 + i*2)*cm), 
    #                 (6*cm - 0.5*cm, A4[1] - (6 + i*2)*cm),
    #                 (6*cm + 0.5*cm, A4[1] - (6 + i*2)*cm)]
    #     c.line(triangle)

    # 再度、テキストを追加
    c.drawString(3*cm, A4[1] - 11*cm, "解答を出力しています...")

    # 枠線を追加
    c.rect(2*cm, A4[1] - 14*cm, 10*cm, 12*cm)

    # PDFを保存
    c.showPage()
    c.save()

# 実際にPDFを生成
output_path = "quiz.pdf"
create_pdf(output_path)

In [9]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
# font設定
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# フォントの登録
font_name = "IPAexGothic"
pdfmetrics.registerFont(TTFont(font_name, "./font/ipaexg00401/ipaexg.ttf"))

c = canvas.Canvas("template.pdf", pagesize=letter)
c.setFont(font_name, 20)
c.drawString(100, 750, "私の名前は _______")
c.save()


In [10]:
import pdfplumber
from reportlab.pdfgen import canvas
from io import BytesIO

text_ex = "宮城県美術館所蔵 絵本原画の世界2022-23	2023/10/7	2023/12/10	https://www.bunka.pref.mie.lg.jp/art-museum/000278004.htm	「こどものとも」は芸術性の高い絵本づくりを目指し、1956年に福音館書店から創刊された月刊絵本です。「こどものとも」の魅力として、洋画や日本画、彫刻、漫画など、さまざまな分野で活躍す る作家たちが絵本づくりを手がけたことがあげられます。作家たちは、ものがたりの表現にふさわしい技法や材料を取りいれながら、これまでにない新しい絵本を生み出しました。　本展では、宮城県美術館の絵本原画コレクションから、「こどものとも」を語る上で欠かせない名作、そして時代をこえて子どもたちに親しまれている絵本の原画を紹介します。"
text_ex = text_ex.split("	")

# テンプレートPDFを開く
with pdfplumber.open("template.pdf") as pdf:
    page = pdf.pages[0]
    text = page.extract_text()

    # プレースホルダを置き換える
    text = text.replace("_______", text_ex[-1])

    # 新しいPDFを生成
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    c.setFont(font_name, 20)
    c.drawString(100, 750, text)
    c.save()
    buffer.seek(0)

    # バイナリデータをファイルに保存
    with open("output.pdf", "wb") as f:
        f.write(buffer.read())


In [15]:
# 自動改行のテスト
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

font_name = "IPAexGothic"
pdfmetrics.registerFont(TTFont(font_name, "./font/ipaexg00401/ipaexg.ttf"))

# ドキュメントの設定
doc = SimpleDocTemplate("sample.pdf", pagesize=letter)

# スタイルの取得
styles = getSampleStyleSheet()

# 日本語フォントの指定
normal_style = styles["Normal"]
normal_style.fontName = font_name
normal_style.fontSize = 20
normal_style.leading = 24

# 長いテキストの定義
long_text = text_ex[-1]

# Paragraphの利用
paragraph = Paragraph(long_text, normal_style)

# ドキュメントに追加
story = [paragraph]
doc.build(story)


In [27]:
# url to qr code
# ! pip install qrcode
import qrcode

QR_STR = 'https://techis.jp'
QR_FILE_NAME = 'techis_hp_url2.png'

#QRコードの設定
qr = qrcode.QRCode(
    version=2, #QRコードのバージョン(1~40)
    error_correction=qrcode.constants.ERROR_CORRECT_H #誤り訂正レベル(L：約7%,M：約15%,Q:約25%,H:約30%)
)
qr.add_data(QR_STR)
qr.make()
img = qr.make_image()
img.save(QR_FILE_NAME)

In [23]:
# 画像のテスト
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

font_name = "IPAexGothic"
pdfmetrics.registerFont(TTFont(font_name, "./font/ipaexg00401/ipaexg.ttf"))

# ドキュメントの設定
doc = SimpleDocTemplate("sample_with_image.pdf", pagesize=letter)

# スタイルの取得
styles = getSampleStyleSheet()
normal_style = styles["Normal"]
normal_style.fontName = font_name
normal_style.fontSize = 20
normal_style.leading = 24

# テキストと画像の追加
paragraph_text = "これはテストの文書です。下に画像が添付されています。"
paragraph = Paragraph(paragraph_text, normal_style)

# 画像を読み込み
img_path = "./image/四日市市立博物館.jpg"
img = Image(img_path, width=400, height=300)  # widthとheightは適宜調整してください

# ドキュメントにテキストと画像を追加
story = [paragraph, img]
doc.build(story)


In [1]:
# 背景画像のテスト
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

font_name = "IPAexGothic"
pdfmetrics.registerFont(TTFont(font_name, "./font/ipaexg00401/ipaexg.ttf"))

output_path = "background_pdf.pdf"
img_path = "./image/四日市市立博物館.jpg"

# Canvasオブジェクトの作成
c = canvas.Canvas(output_path, pagesize=letter)

# 画像をページ全体の背景として描画
c.drawInlineImage(img_path, 0, 0, width=letter[0], height=letter[1])

# 背景の上にテキストを描画
c.setFont(font_name, 20)
c.drawString(inch, letter[1] - inch, "これは背景画像の上のテキストです。")

# PDFを保存
c.save()

ModuleNotFoundError: No module named 'reportlab'